In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath("")
    os.chdir(os.path.dirname(file_path))
    print(file_path)


/root/code_Bao/stock_price_4_fun/notebook


In [2]:

# Data Handling
import pandas as pd
import numpy as np

# Data Visualization
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
from IPython.display import display
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Statistics & Mathematics
import scipy.stats as stats
import statsmodels as sm
from scipy.stats import shapiro, skew
import math
import mplfinance as mpf
import matplotlib.pyplot as plt

# Hiding warnings 
import warnings
warnings.filterwarnings("ignore")

from vnstock import *  # import all functions
from src.utils import *
from src.support_resist import SupportResistFinding
import urllib3

urllib3.disable_warnings()


In [3]:
symbol = "VND"


In [4]:
current_date = datetime.now().strftime("%Y-%m-%d")


In [5]:
time_col = "time"
float_cols = ["open", "high", "low", 'close', "volume"]
cat_cols = ["ticker"]


In [6]:
df = stock_historical_data(symbol, "2007-01-01" ,current_date, "1D", type ='stock')
df = convert_data_type(df, [time_col], float_cols, cat_cols)


 # VOLATILITY BASED

In [7]:
df['Returns'] = df['close'].pct_change(1)


In [8]:
T = 20 # Time period for computing the standard deviation
df['annualized_vol'] = np.round(df['Returns'].rolling(T).std() * np.sqrt(252), 2)
df['monthly_vol'] = (df['annualized_vol']) / np.sqrt(12) 
df['weekly_vol'] = (df['annualized_vol']) / np.sqrt(52) 


In [9]:
df.head()


,time,open,high,low,close,volume,ticker,Returns,annualized_vol,monthly_vol,weekly_vol
0,2012-03-20,1420.0,1490.0,1420.0,1480.0,4226200.0,VND,NaN,NaN,NaN,NaN
1,2012-03-21,1550.0,1580.0,1510.0,1560.0,5782900.0,VND,0.054054,NaN,NaN,NaN
2,2012-03-22,1530.0,1580.0,1490.0,1550.0,4674100.0,VND,-0.006410,NaN,NaN,NaN
3,2012-03-23,1530.0,1650.0,1520.0,1580.0,6964400.0,VND,0.019355,NaN,NaN,NaN
4,2012-03-26,1640.0,1650.0,1570.0,1580.0,4366400.0,VND,0.000000,NaN,NaN,NaN


In [10]:
def plot_volatility(symbol, volatility_col = 'annualized_vol'):
    # Creating a line plot to visualize historical volatility
    lineplot = go.Scatter(
        x=df.time,
        y=df[volatility_col] * 100, # Annualized Volatility in %
        mode='lines',
        line=dict(color='darkblue', width=2.5),
        name = 'Annualized Volatility (%)')

    layout = go.Layout(
        title={'text': f'<b>Annualized Volatility for {symbol} <br><i><sub>S&P 500</sub></i></b>',
            'x': 0.035, 'xanchor': 'left'},
                yaxis = dict(title = '<b>Annualized Volatility (%) </b>'), 
                xaxis = dict(title = '<b>Date</b>'),
        template='seaborn',
        height = 450, width = 1000,
        showlegend=True,
        plot_bgcolor = '#F6F5F5',
        paper_bgcolor = '#F6F5F5',
        xaxis_rangeslider_visible=False)

    fig = go.Figure(data=[lineplot], layout=layout)

    fig.show()


In [11]:
plot_volatility(symbol, 'annualized_vol')


In [12]:
plot_volatility(symbol, 'monthly_vol')


In [13]:
plot_volatility(symbol, 'weekly_vol')


In [14]:
df.set_index('time', inplace = True)


In [15]:
# Yearly forecasting

reference_year = "2022" # Forecasting levels for the (referance year + 1)

def calulate_hl_band(reference_year = '2023', volatility_col = "annualized_vol", price_col = "close" ):
    
    High_Band_1std =  df.loc[reference_year][volatility_col][-1]*df.loc[reference_year][price_col][-1] + df.loc[reference_year][price_col][-1]
    Low_Band_1std =  df.loc[reference_year][price_col][-1] - df.loc[reference_year][volatility_col][-1]*df.loc[reference_year][price_col][-1]

    High_Band_2std =  2*df.loc[reference_year][volatility_col][-1]*df.loc[reference_year][price_col][-1] + df.loc[reference_year][price_col][-1]
    Low_Band_2std =  df.loc[reference_year][price_col][-1] - 2*df.loc[reference_year][volatility_col][-1]*df.loc[reference_year][price_col][-1]

    High_Band_3std =  3*df.loc[reference_year][volatility_col][-1]*df.loc[reference_year][price_col][-1] + df.loc[reference_year][price_col][-1]
    Low_Band_3std =  df.loc[reference_year][price_col][-1] - 3*df.loc[reference_year][volatility_col][-1]*df.loc[reference_year][price_col][-1]
    return High_Band_3std, High_Band_2std, High_Band_1std, Low_Band_1std, Low_Band_2std, Low_Band_3std


In [16]:
High_Band_3std, High_Band_2std, High_Band_1std, Low_Band_1std, Low_Band_2std, Low_Band_3std = calulate_hl_band(reference_year=reference_year, volatility_col = 'weekly_vol')


In [17]:
print(f'\nVolatility-Based Supply and Demand Levels for {symbol} in {int(reference_year) + 1}\n')
print(f'Supply Level 3σ: {High_Band_3std.round(2)}\n')
print(f'Supply Level 2σ: {High_Band_2std.round(2)}\n')
print(f'Supply Level 1σ: {High_Band_1std.round(2)}\n')
print('-' * 65, '\n')
print(f'Demand Level 1σ: {Low_Band_1std.round(2)}\n')
print(f'Demand Level 2σ: {Low_Band_2std.round(2)}\n')
print(f'Demand Level 3σ: {Low_Band_3std.round(2)}\n')



Volatility-Based Supply and Demand Levels for VND in 2023

Supply Level 3σ: 17094.46

Supply Level 2σ: 15896.3

Supply Level 1σ: 14698.15

----------------------------------------------------------------- 

Demand Level 1σ: 12301.85

Demand Level 2σ: 11103.7

Demand Level 3σ: 9905.54



In [18]:
# Candlestick chart
candlestick = go.Candlestick(x = df.loc[str(int(reference_year) + 1)].index,
                             open = df.loc[str(int(reference_year) + 1)]['open'],
                             high = df.loc[str(int(reference_year) + 1)]['high'],
                             low = df.loc[str(int(reference_year) + 1)]['low'],
                             close = df.loc[str(int(reference_year) + 1)]['close'],
                             increasing = dict(line=dict(color = 'red')),
                             decreasing = dict(line=dict(color = 'black')),
                             name = 'Candlesticks')


# Defining layout
layout = go.Layout(title = {'text': '<b>Volatility-Based Supply and Demand Levels<br><sup>&nbsp;&nbsp;&nbsp;&nbsp;<i>Annualized Forecast</i></sup></b>',
                            'x': .035, 'xanchor': 'left'},
                   yaxis = dict(title = '<b>Price (USD)</b>',
                               tickfont=dict(size=16)),
                   xaxis = dict(title = '<b>Date</b>'),
                   template = 'seaborn',
                   plot_bgcolor = '#F6F5F5',
                   paper_bgcolor = '#F6F5F5',
                   height = 850, width = 1000,
                   showlegend=False,
                   xaxis_rangeslider_visible = False)

# Defining figure
fig = go.Figure(data = [candlestick], layout = layout)

# Removing empty spaces (non-trading days)
dt_all = pd.date_range(start = df.index[0],
                       end = df.index[-1],
                       freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df.index]
dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)

# Adding Supply and Demans Lines

# 1σ
fig.add_hline(y = High_Band_1std, line_width = 2, line_dash = "dot", line_color = "green")
fig.add_hline(y = Low_Band_1std, line_width = 2, line_dash = "dot", line_color = "red")

# 2σ
fig.add_hline(y = High_Band_2std, line_width = 4, line_dash = "dash", line_color = "green")
fig.add_hline(y = Low_Band_2std, line_width = 4, line_dash = "dash", line_color = "red")

# 3σ
fig.add_hline(y = High_Band_3std, line_width = 6, line_dash = "dashdot", line_color = "green")
fig.add_hline(y = Low_Band_3std, line_width = 6, line_dash = "dashdot", line_color = "red")

# Showing plot
fig.show()


 # CANDLE STICKS

In [19]:
sr_finding = SupportResistFinding(symbol, accuracy=7, percent_threshold=3)

In [20]:
levels = sr_finding.calculate_support_resistance_band()
len(levels)

9

In [21]:
sr_finding.plot_candlestick_with_levels()

In [22]:
result = sr_finding.find_closest_support_resist(current_price=16000)

In [23]:
result

[14800.0, 17350.0]